# Zero-shot Labeling with OpenCLIP

Demonstrate zero-shot scoring of tissue patches using CLIP-like models. This is useful for rapid prototyping without labeled data.

In [ ]:
# Optional install: open_clip_torch
import importlib
if importlib.util.find_spec('open_clip') is None:
    get_ipython().system('pip -q install open_clip_torch')
import torch, open_clip, torchvision.transforms as T
from PIL import Image
import io, requests, numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device=device)
tokenizer = open_clip.get_tokenizer('ViT-B-32')
# Load a sample image
url = 'https://picsum.photos/seed/histo2/256'
try:
    img = Image.open(io.BytesIO(requests.get(url, timeout=5).content)).convert('RGB')
except Exception:
    img = Image.fromarray((np.random.rand(256,256,3)*255).astype('uint8'), 'RGB')
image = preprocess(img).unsqueeze(0).to(device)
texts = ['tumor tissue', 'normal epithelium', 'lymphocytes', 'stroma']
text_tokens = tokenizer(texts).to(device)
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    logits = (100.0 * image_features @ text_features.T).softmax(dim=-1).squeeze().cpu().numpy()
for label, p in zip(texts, logits):
    print(f'{label:>20}: {p:.3f}')